In [1]:
import graph_tool.all as gt
import networkx as nx
import sys
from itertools import combinations
import math
import numpy as np
from core_periphery_sbm import core_periphery as cp


# Convert a graph-tool graph to a NetworkX graph
def graph_tool_to_networkx(gt_graph):
    nx_graph = nx.Graph() if not gt_graph.is_directed() else nx.DiGraph()

    # Add nodes
    for v in gt_graph.vertices():
        nx_graph.add_node(int(v))

    # Add edges
    for e in gt_graph.edges():
        nx_graph.add_edge(int(e.source()), int(e.target()))

    return nx_graph

# Load Les Mis graph from NetworkX
G = graph_tool_to_networkx(gt.collection.ns["terrorists_911"])

print("---------GALLAGHER'S METHOD---------")
# Initialize and infer Hub-and-Spoke model
hubspoke = cp.HubSpokeCorePeriphery(n_gibbs=100, n_mcmc=100)  # Using a fixed number of MCMC steps for the whole graph
hubspoke.infer(G)

# Initialize and infer Layered model
layered = cp.LayeredCorePeriphery(n_layers=3, n_gibbs=100, n_mcmc=100)  # Using a fixed number of MCMC steps for the whole graph
layered.infer(G)

# Get node-to-group assignments for both models
node2label_hs = hubspoke.get_labels(prob=False, return_dict=True)
node2label_l = layered.get_labels(prob=False, return_dict=True)

# Calculate P(A | k, g)
def calculate_p_a_k_g(graph, node2label):
    """
    Calculate P(A | k, g) for a graph and node-to-layer mapping.

    Args:
        graph (nx.Graph): The input graph (NetworkX format).
        node2label (dict): Mapping of nodes to their layer assignments.

    Returns:
        float: The value of P(A | k, g).
    """
    # Number of layers (k)
    k = len(set(node2label.values()))

    # Initialize m_r and t_r for each group
    m = [0] * k  # Number of edges within each group
    t = [0] * k  # Number of possible pairs within each group

    # Reverse the node-to-layer mapping to group nodes by layers
    layer_to_nodes = {r: [] for r in range(k)}
    for node, layer in node2label.items():
        layer_to_nodes[layer].append(node)

    # Calculate m_r and t_r for each group
    for r in range(k):
        nodes_in_layer = layer_to_nodes[r]
        if len(nodes_in_layer) < 2:
            continue  # Skip groups with fewer than 2 nodes

        # Count edges within the group
        m[r] = sum(1 for u, v in combinations(nodes_in_layer, 2) if graph.has_edge(u, v))

        # Count total possible pairs within the group
        t[r] = len(nodes_in_layer) * (len(nodes_in_layer) - 1) // 2

    # Calculate P(A | k, g) using the formula
    p_a_k_g = 1.0
    for r in range(k):
        if t[r] > 0:  # Avoid division by zero for empty groups
            p_a_k_g *= (math.factorial(m[r]) * math.factorial(t[r] - m[r])) / math.factorial(t[r] + 1)

    return p_a_k_g

# Print probabilities for Hub-and-Spoke model
node2probs_hs = hubspoke.get_labels(prob=True, return_dict=True)
print("Node to probabilities (HubSpoke):", node2probs_hs)

# Print core and periphery nodes for Hub-and-Spoke model
core_nodes_hs = [node for node, group in node2label_hs.items() if group == 0]  # Assuming core is group 0
periphery_nodes_hs = [node for node, group in node2label_hs.items() if group != 0]
print("Core Nodes (HubSpoke):", core_nodes_hs)
print("Periphery Nodes (HubSpoke):", periphery_nodes_hs)

# Print probabilities for Layered model
node2probs_l = layered.get_labels(prob=True, return_dict=False)
print("Node to probabilities (Layered):", node2probs_l)

# Count and print number of layers detected in the Layered model
num_layers = len(set(node2label_l.values()))
print(f"Number of layers detected in the Layered structure: {num_layers}")

# Identify core and periphery nodes in the Layered model
layer_to_nodes = {layer: [] for layer in range(num_layers)}
for node, layer in node2label_l.items():
    layer_to_nodes[layer].append(node)

# Print core and periphery nodes for Layered model with layer numbers
for layer, nodes in layer_to_nodes.items():
    if layer == 0:
        print(f"Core Nodes (Layer {layer + 1}):", nodes)
    else:
        print(f"Periphery Nodes (Layer {layer + 1}):", nodes)

# Calculate P(A | k, g) for Hub-and-Spoke structure
p_a_k_g_hs = calculate_p_a_k_g(G, node2label_hs)
print("P(A | k, g) for Hub-and-Spoke Structure:", p_a_k_g_hs)

# Calculate P(A | k, g) for Layered structure
p_a_k_g_l = calculate_p_a_k_g(G, node2label_l)
print("P(A | k, g) for Layered Structure:", p_a_k_g_l)

# Determine which structure is more suitable
if p_a_k_g_hs > p_a_k_g_l:
    print("Hub-and-Spoke structure is more suitable for this network.")
else:
    print("Layered structure is more suitable for this network.")
print("----------------------------------------------")
print("---------BOGATTI AND EVERETT'S METHOD---------")
# Function to compute core-periphery assignments using Borgatti-Everett method
def borgatti_everett_core_periphery(graph):
    nodes = list(graph.nodes())
    adjacency_matrix = nx.to_numpy_array(graph)
    
    # Start with random assignments of coreness (C values) for each node
    coreness = np.random.rand(len(nodes))
    
    def pattern_matrix(c):
        return np.outer(c, c)

    def objective_function(c):
        p_matrix = pattern_matrix(c)
        correlation = np.corrcoef(adjacency_matrix[np.triu_indices(len(nodes), k=1)],
                                  p_matrix[np.triu_indices(len(nodes), k=1)])[0, 1]
        return -correlation  # Minimize negative correlation

    # Optimize coreness values using gradient-free optimization (e.g., scipy.optimize)
    from scipy.optimize import minimize
    result = minimize(objective_function, coreness, bounds=[(0, 1)] * len(nodes), method="L-BFGS-B")
    
    final_coreness = result.x
    return dict(zip(nodes, final_coreness))

# Function to calculate P(A | k, g)
def calculate_p_a_k_g(graph, node2coreness):
    nodes = list(graph.nodes())
    adjacency_matrix = nx.to_numpy_array(graph)
    pattern_matrix = np.outer(list(node2coreness.values()), list(node2coreness.values()))

    m = np.sum(adjacency_matrix * pattern_matrix)
    t = np.sum(pattern_matrix)

    if t > 0:
        return (math.factorial(int(m)) * math.factorial(int(t - m))) / math.factorial(int(t + 1))
    return 0

# Load the terrorist dataset
G_gt = gt.collection.ns["terrorists_911"]
G = graph_tool_to_networkx(G_gt)

# Apply Borgatti-Everett core-periphery method
node2coreness = borgatti_everett_core_periphery(G)

# Threshold coreness to identify core and periphery nodes
threshold = np.median(list(node2coreness.values()))
print(f"Median Coreness Threshold: {threshold}")
core_nodes = [node for node, coreness in node2coreness.items() if coreness >= threshold]
periphery_nodes = [node for node, coreness in node2coreness.items() if coreness < threshold]

# Print results
print("Node Coreness Values:", node2coreness)
print("Core Nodes:", core_nodes)
print("Periphery Nodes:", periphery_nodes)

# Calculate P(A | k, g)
p_a_k_g = calculate_p_a_k_g(G, node2coreness)
print("P(A | k, g) for Borgatti-Everett Method:", p_a_k_g)


/home/thanos/Documents/leiden/S1/SNA/SNA-14B/core_periphery_sbm/core_periphery.py:344: SyntaxWarning: invalid escape sequence '\m'
  """
/home/thanos/Documents/leiden/S1/SNA/SNA-14B/core_periphery_sbm/core_periphery.py:415: SyntaxWarning: invalid escape sequence '\s'
  """
/home/thanos/Documents/leiden/S1/SNA/SNA-14B/core_periphery_sbm/core_periphery.py:550: SyntaxWarning: invalid escape sequence '\m'
  """
/home/thanos/Documents/leiden/S1/SNA/SNA-14B/core_periphery_sbm/core_periphery.py:743: SyntaxWarning: invalid escape sequence '\m'
  """
/home/thanos/Documents/leiden/S1/SNA/SNA-14B/core_periphery_sbm/inference.py:63: SyntaxWarning: invalid escape sequence '\m'
  """
/home/thanos/Documents/leiden/S1/SNA/SNA-14B/core_periphery_sbm/inference.py:284: SyntaxWarning: invalid escape sequence '\m'
  """
/home/thanos/Documents/leiden/S1/SNA/SNA-14B/core_periphery_sbm/inference.py:308: SyntaxWarning: invalid escape sequence '\m'
  """


---------GALLAGHER'S METHOD---------
Node to probabilities (HubSpoke): {0: array([0.26, 0.74]), 1: array([0.82, 0.18]), 2: array([0.27, 0.73]), 3: array([0.26, 0.74]), 4: array([0.26, 0.74]), 5: array([0.28, 0.72]), 6: array([0.18, 0.82]), 7: array([0.36, 0.64]), 8: array([0.32, 0.68]), 9: array([0.53, 0.47]), 10: array([0.36, 0.64]), 11: array([0.88, 0.12]), 12: array([0.4, 0.6]), 13: array([0.38, 0.62]), 14: array([0.26, 0.74]), 15: array([0.54, 0.46]), 16: array([0.27, 0.73]), 17: array([0.66, 0.34]), 18: array([0.35, 0.65]), 19: array([0.25, 0.75]), 20: array([0.25, 0.75]), 21: array([0.27, 0.73]), 22: array([0.35, 0.65]), 23: array([0.28, 0.72]), 24: array([0.87, 0.13]), 25: array([0.29, 0.71]), 26: array([0.45, 0.55]), 27: array([0.43, 0.57]), 28: array([0.52, 0.48]), 29: array([0.72, 0.28]), 30: array([0.75, 0.25]), 31: array([0.86, 0.14]), 32: array([0.21, 0.79]), 33: array([0.25, 0.75]), 34: array([0.28, 0.72]), 35: array([0.8, 0.2]), 36: array([0.57, 0.43]), 37: array([0.39, 